In [1]:
from bs4 import BeautifulSoup
import requests

strana = 1
# Získám HTML kód webu
response = requests.get(f'https://www.sreality.cz/hledani/pronajem/byty?region=Boskovice&region-id=6137&region-typ=municipality&lat-max=49.51874510791316&lat-min=49.43866396851723&lon-max=16.68308258056641&lon-min=16.602230072021488').text


In [2]:
#Z HTML kódu udělám BeautifulSoup objekt
soup_byty = BeautifulSoup (response, 'html.parser')
# Získám HTML kód pouze potřebné části
unordered_list = soup_byty.find('ul', 'MuiGrid2-root MuiGrid2-container MuiGrid2-direction-xs-row css-1fesoy9')
# Získám HTML kód všech bytů do jednoho listu
vsechny_byty = unordered_list.find_all('li')
list_bytu = []
# Teď budu procházet jednotlivé byty v listu a hledat odkaz (<a> tag)
for byt in vsechny_byty:
    print(byt)
    if byt.find('a','MuiTypography-root' ) is None:
        byt_odkaz = None
    else:
        byt_odkaz = byt.find('a', 'MuiTypography-root')['href']
        print(f'Odkaz na byt: {byt_odkaz}')
   
   # Rozměry bytu v m2
    if byt.find('p', 'css-d7upve') is None:
        rozmery_bytu = None
    else:
        rozmery_bytu = byt.find('p', 'css-d7upve').text.replace('SLEVA: ', '').split(' ')[3]
        print(f'Rozměry bytu: {rozmery_bytu}')
    
    # Dispozice bytu
    if byt.find('p', 'css-d7upve') is None:
        dispozice_bytu = None
    else:
        dispozice_bytu = byt.find('p', 'css-d7upve').text.replace('SLEVA: ', '').split(' ')[2]
        print(f'Dispozice bytu: {byt.find('p', 'css-d7upve').text.split(' ')[1]}')
    
    # Cena bytu
    if byt.find('p', 'css-ca9wwd') is None:
        cena_bytu = None

    else:
        cena_bytu = byt.find('p', 'css-ca9wwd').text
        print(f'Cena bytu: {cena_bytu}')
    
    #Lokace bytu
    if byt.find('p', 'css-d7upve') is None:
        lokace = None
    else: 
        lokace = byt.find_all('p', 'css-d7upve')[1].text

    list_bytu.append({
        'Rozmery bytu': rozmery_bytu,
        'Dispozice bytu': dispozice_bytu,
        'Cena bytu': cena_bytu,
        'Lokace Bytu': lokace
    })






<li class="MuiGrid2-root MuiGrid2-direction-xs-row css-1oxc29r" id="estate-list-item-2565083980"><style data-emotion="css x11yz8">.css-x11yz8{-webkit-text-decoration:underline;text-decoration:underline;text-decoration-color:var(--Link-underlineColor);--Link-underlineColor:rgba(25, 118, 210, 0.4);scroll-margin-top:16px;cursor:pointer;-webkit-text-decoration:none;text-decoration:none;color:#626D82;width:100%;}.css-x11yz8:hover{text-decoration-color:inherit;}.css-x11yz8:focus-visible{outline:0;box-shadow:0 0 0.188rem 0.063rem #03a9f4;}.css-x11yz8:hover{color:#232B3A;}.css-x11yz8:focus-visible{box-shadow:none;outline:#03a9f4 auto 1px;}.css-x11yz8:visited p{color:#5E2F81;}</style><style data-emotion="css 1s6ohwi">.css-1s6ohwi{margin:0;font:inherit;line-height:inherit;letter-spacing:inherit;color:#1976d2;-webkit-text-decoration:underline;text-decoration:underline;text-decoration-color:var(--Link-underlineColor);--Link-underlineColor:rgba(25, 118, 210, 0.4);scroll-margin-top:16px;cursor:point

In [3]:
# Zde jsem využil AI:
from bs4 import BeautifulSoup
import pandas as pd

# 1. Z HTML kódu udělám BeautifulSoup objekt
# Předpokládáme, že 'response' je proměnná s načteným HTML kódem
soup_byty = BeautifulSoup(response, 'html.parser')

# 2. Najdeme všechny položky seznamu (inzeráty i reklamy)
vsechny_polozky = soup_byty.find_all('li')
list_bytu = []

# 3. Procházení jednotlivých položek
for polozka in vsechny_polozky:
    # --- Inicializace proměnných pro každý řádek ---
    rozmery_bytu = "Neuvedeno"
    dispozice_bytu = "Neuvedeno"
    cena_bytu = "Neuvedeno"
    ulice = "Neuvedeno"
    obec = "Neuvedeno"
    
    # Získáme všechny textové části z inzerátu (odstavce i spany)
    texty = [t.get_text(strip=True) for t in polozka.find_all(['p', 'span'])]
    
    # Pokud v položce nejsou žádné texty, rovnou ji přeskočíme (je to prázdný kontejner)
    if not texty:
        continue

    # --- Analýza nalezených textů ---
    for text in texty:
        # Hledáme cenu (obsahuje Kč)
        if 'Kč' in text and cena_bytu == "Neuvedeno":
            cena_bytu = text
        
        # Hledáme parametry (obsahují m²)
        elif 'm²' in text:
            casti = text.replace('SLEVA: ', '').split()
            for i, slovo in enumerate(casti):
                if '+' in slovo or slovo.lower() == 'atypické':
                    dispozice_bytu = slovo
                if slovo.replace(',', '').isdigit() and i + 1 < len(casti) and 'm' in casti[i+1]:
                    rozmery_bytu = f"{slovo} m²"

        # Hledáme lokaci (text s čárkou, co není cena ani rozměr)
        elif ',' in text and 'm²' not in text and 'Kč' not in text and len(text) > 5:
            casti_lokace = text.split(',')
            ulice = casti_lokace[0].strip()
            obec = casti_lokace[1].strip()
        
        # Pokud inzerát nemá čárku, ale je to text obce (často jen jeden název)
        elif len(text) < 40 and obec == "Neuvedeno" and 'm²' not in text and 'Kč' not in text and 'byt' not in text.lower():
            if len(text) > 3: # Ochrana před příliš krátkými úlomky kódu
                obec = text.strip()

    # --- FILTR PROTI REKLAMÁM ---
    # Inzerát uložíme, jen pokud má rozměry nebo cenu. 
    # Reklamy tyto údaje v tomto formátu nemají, takže se do listu nedostanou.
    if cena_bytu != "Neuvedeno" or rozmery_bytu != "Neuvedeno":
        # Ještě jedna pojistka: pokud jsme našli cenu, ale ne obec, je to pravděpodobně reklama
        if obec != "Neuvedeno":
            list_bytu.append({
                'Rozmery bytu': rozmery_bytu,
                'Dispozice bytu': dispozice_bytu,
                'Cena bytu': cena_bytu,
                'Ulice': ulice,
                'Obec': obec
            })

In [6]:
# Následně zpracuju zjištěná data do tabulky
import pandas as pd
df = pd.DataFrame(list_bytu)

df.head(10)

,Rozmery bytu,Dispozice bytu,Cena bytu,Ulice,Obec
0,61 m²,2+kk,15 000 Kč/měsíc,Hybešova,Boskovice
1,43 m²,1+kk,13 900 Kč/měsíc,Otakara Kubína,Boskovice
2,56 m²,2+kk,17 000 Kč/měsíc,Hybešova,Boskovice
3,60 m²,2+kk,14 000 Kč/měsíc,Štefanikova,Boskovice
4,150 m²,4+1,15 000 Kč/měsíc,Masarykovo náměstí,Boskovice
5,80 m²,2+kk,17 000 Kč/měsíc,Svatopluka Čecha,Boskovice


In [8]:
df.to_csv('{MarekVašíček}_surova_data.csv', index=False, encoding='utf-8-sig', sep=';')